# Azure Cognitive Services Face サービス

- [Face のドキュメント](https://docs.microsoft.com/ja-jp/azure/cognitive-services/face/)

- [Azure Face サービスとは](https://docs.microsoft.com/ja-jp/azure/cognitive-services/face/overview)

In [499]:
import asyncio
import io
import glob
import os
import sys
import time
import uuid
import requests
from urllib.parse import urlparse
from io import BytesIO
from PIL import Image, ImageDraw
# Azure SDK for Pythonに含まれるFaceAPIを取り扱うクラス
# FaceClientはdetectやverificationなどの各種Face API機能を扱うためのクラス
from azure.cognitiveservices.vision.face import FaceClient
# Cognitive Serviceの認証をAPIキーを通じて行うためのクラス
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.face.models import TrainingStatusType, Person, SnapshotObjectType, OperationStatusType

- [FaceClient class
](https://docs.microsoft.com/ja-jp/python/api/azure-cognitiveservices-vision-face/azure.cognitiveservices.vision.face.faceclient?view=azure-python)

In [500]:
KEY = os.environ['FACE_SUBSCRIPTION_KEY']
ENDPOINT = os.environ['FACE_ENDPOINT']

# FaceAPIクライアントのインスタンス作成
face_client = FaceClient(ENDPOINT, CognitiveServicesCredentials(KEY))

In [501]:
facekeys = []
facepaths = []

for dirname, _, filenames in os.walk('images'):
    for filename in filenames:
        facekeys.append(filename.strip('.png'))
        facepaths.append(os.path.join(dirname, filename))
        # print(filename.strip('.png')+' = '+'\''+os.path.join(dirname, filename)+'\'')
        
face_path_dict = dict(zip(facekeys, facepaths))
face_path_dict

{'hanzawa1': 'images/hanzawa1.png',
 'kurosaki1': 'images/kurosaki1.png',
 'owada1': 'images/owada1.png',
 'hanzawa2': 'images/hanzawa2.png',
 'kurosaki2': 'images/kurosaki2.png',
 'owada3': 'images/owada3.png',
 'owada2': 'images/owada2.png',
 'kurosaki3': 'images/kurosaki3.png',
 'hanzawa3': 'images/hanzawa3.png'}

- [detect_with_stream](https://docs.microsoft.com/ja-jp/python/api/azure-cognitiveservices-vision-face/azure.cognitiveservices.vision.face.operations.faceoperations?view=azure-python#detect-with-stream-image--return-face-id-true--return-face-landmarks-false--return-face-attributes-none--recognition-model--recognition-01---return-recognition-model-false--detection-model--detection-01---custom-headers-none--raw-false--callback-none----operation-config-)

In [502]:
# 読み込んだ画像を送信して顔検出
detected_faces = []
keys = []

for key, face in face_path_dict.items():
    with open(face, 'rb') as image: # バイナリモードrbで読み込み
        keys.append(key)
        detected_faces.append(
            face_client.face.detect_with_stream(
                image,
                return_face_attributes=['age', 'gender', 'smile', 'glasses','emotion']))

In [503]:
detected_faces_dict = dict(zip(keys, detected_faces))
detected_faces_dict

{'hanzawa1': [<azure.cognitiveservices.vision.face.models._models_py3.DetectedFace at 0x7ffb21c01520>],
 'kurosaki1': [<azure.cognitiveservices.vision.face.models._models_py3.DetectedFace at 0x7ffb9c743bb0>],
 'owada1': [<azure.cognitiveservices.vision.face.models._models_py3.DetectedFace at 0x7ffb9aacf580>],
 'hanzawa2': [<azure.cognitiveservices.vision.face.models._models_py3.DetectedFace at 0x7ffb9ade19a0>],
 'kurosaki2': [<azure.cognitiveservices.vision.face.models._models_py3.DetectedFace at 0x7ff9141f6a90>],
 'owada3': [<azure.cognitiveservices.vision.face.models._models_py3.DetectedFace at 0x7ffb9ade1760>],
 'owada2': [<azure.cognitiveservices.vision.face.models._models_py3.DetectedFace at 0x7ff9144e2190>],
 'kurosaki3': [<azure.cognitiveservices.vision.face.models._models_py3.DetectedFace at 0x7ff9c3d2da00>],
 'hanzawa3': [<azure.cognitiveservices.vision.face.models._models_py3.DetectedFace at 0x7ff9c3dc8ee0>]}

In [504]:
# hanzawa1を指定
print(detected_faces_dict['hanzawa1'])

In [505]:
print(detected_faces_dict['hanzawa1'][0])

{'additional_properties': {}, 'face_id': '14a77c62-d264-4c87-bd2a-1fe61e3ed57b', 'recognition_model': None, 'face_rectangle': <azure.cognitiveservices.vision.face.models._models_py3.FaceRectangle object at 0x7ffb9c425df0>, 'face_landmarks': None, 'face_attributes': <azure.cognitiveservices.vision.face.models._models_py3.FaceAttributes object at 0x7ff91439bbb0>}


In [506]:
# hanzawa1の'age', 'gender', 'smile', 'glasses','emotion'
print(detected_faces_dict['hanzawa1'][0].face_attributes)

{'additional_properties': {}, 'age': 37.0, 'gender': <Gender.male: 'male'>, 'smile': 0.002, 'facial_hair': None, 'glasses': <GlassesType.no_glasses: 'noGlasses'>, 'head_pose': None, 'emotion': <azure.cognitiveservices.vision.face.models._models_py3.Emotion object at 0x7ff91409a880>, 'hair': None, 'makeup': None, 'occlusion': None, 'accessories': None, 'blur': None, 'exposure': None, 'noise': None}


In [507]:
# owada1の'age', 'gender', 'smile', 'glasses','emotion'
print(detected_faces_dict['owada1'][0].face_attributes)

{'additional_properties': {}, 'age': 48.0, 'gender': <Gender.male: 'male'>, 'smile': 0.982, 'facial_hair': None, 'glasses': <GlassesType.no_glasses: 'noGlasses'>, 'head_pose': None, 'emotion': <azure.cognitiveservices.vision.face.models._models_py3.Emotion object at 0x7ff9144e2640>, 'hair': None, 'makeup': None, 'occlusion': None, 'accessories': None, 'blur': None, 'exposure': None, 'noise': None}


## 二つの画像が同じ人物かそうでないかを判定する「Face Verify」

### モデルテスト

In [508]:
# 顔検出結果のFaceIDを取得する
n = len(detected_faces)
ids = []

for i in range(0, n):
    ids.append(detected_faces[i][0].face_id)

face_id_dict = dict(zip(dict_all.keys(), ids))
face_id_dict

{'hanzawa1': '14a77c62-d264-4c87-bd2a-1fe61e3ed57b',
 'kurosaki1': '0d45d73f-b29f-48b0-84ff-8e943feb6aba',
 'owada1': '33626298-e410-4bf6-8b5e-e75887b10c93',
 'hanzawa2': 'aa051851-5ee3-4257-8e2a-6bbb9730fb2c',
 'kurosaki2': 'ce2cf59a-d6bc-42d5-bac1-ef4ffc579558',
 'owada3': '96be1e47-e5d2-4564-b102-15adf6c9702a',
 'owada2': 'c7f1ed7f-2242-4480-af98-b26fb9d27aba',
 'kurosaki3': '9ed35e62-4593-4a58-84bb-7a5f6ed4fc91',
 'hanzawa3': '1d3ea00a-481f-43e6-ba13-0396468115be'}

In [509]:
face_id_dict['hanzawa1']

'14a77c62-d264-4c87-bd2a-1fe61e3ed57b'

In [510]:
# 同じ人物かを判定する（半沢と大和田）
verified = face_client.face.verify_face_to_face(face_id_dict['hanzawa1'], face_id_dict['owada1'])
print(verified)

{'additional_properties': {}, 'is_identical': False, 'confidence': 0.16937}


In [511]:
# 同じ人物かを判定する（半沢と黒崎）
verified = face_client.face.verify_face_to_face(face_id_dict['hanzawa1'], face_id_dict['kurosaki1'])
print(verified)

{'additional_properties': {}, 'is_identical': False, 'confidence': 0.28355}


In [512]:
# 同じ人物かを判定する（同じ画像を入れてみる）
verified = face_client.face.verify_face_to_face(face_id_dict['hanzawa1'], face_id_dict['hanzawa1'])
print(verified)

{'additional_properties': {}, 'is_identical': True, 'confidence': 1.0}


In [513]:
# 同じ人物かを判定する（表情の似た同じ人物の画像を入れてみる）
verified = face_client.face.verify_face_to_face(face_id_dict['hanzawa1'], face_id_dict['hanzawa2'])
print(verified)

{'additional_properties': {}, 'is_identical': True, 'confidence': 0.6732}


In [514]:
# 同じ人物かを判定する（表情の違う同じ人物の画像を入れてみる）
verified = face_client.face.verify_face_to_face(face_id_dict['hanzawa1'], face_id_dict['hanzawa3'])
print(verified)

{'additional_properties': {}, 'is_identical': False, 'confidence': 0.43218}


### 学習用Person Groupの作成

- [Create a new person group with specified personGroupId](https://docs.microsoft.com/ja-jp/python/api/azure-cognitiveservices-vision-face/azure.cognitiveservices.vision.face.operations.persongroupoperations?view=azure-python#create-person-group-id--name-none--user-data-none--recognition-model--recognition-01---custom-headers-none--raw-false----operation-config-)

In [515]:
# Person Group IDを指定
person_group_id = 'hanzawa_group'

In [516]:
# person_group初期化（念の為）
face_client.person_group.delete(person_group_id)

In [517]:
# Person Groupを作成する
face_client.person_group.create(
    person_group_id=person_group_id, 
    name='Person Group for LINE Bot'
)

In [518]:
# Person Groupを確認
hanzawa_group = face_client.person_group.get(person_group_id)
print(hanzawa_group)

{'additional_properties': {}, 'name': 'Person Group for LINE Bot', 'user_data': None, 'recognition_model': None, 'person_group_id': 'hanzawa_group'}


### 学習用Personの作成

#### 顔を入れるための箱を作成

- [create(person_group_id, name=None, user_data=None, custom_headers=None, raw=False, **operation_config)](https://docs.microsoft.com/ja-jp/python/api/azure-cognitiveservices-vision-face/azure.cognitiveservices.vision.face.operations.persongrouppersonoperations?view=azure-python#create-person-group-id--name-none--user-data-none--custom-headers-none--raw-false----operation-config-)

In [519]:
# 名前を登録
hanzawa_name = 'hanzawa_naoki'
owada_name = 'owada_akira'
kurosaki_name = 'kurosaki_syunichi'

In [520]:
hanzawa_group.person_group_id

'hanzawa_group'

In [521]:
# Personを登録
hanzawa = face_client.person_group_person.create(
    person_group_id = hanzawa_group.person_group_id, # PersonGroupのIDを指定
    name = hanzawa_name # 登録するPersonの名前を指定
)

owada = face_client.person_group_person.create(
    person_group_id = hanzawa_group.person_group_id, # PersonGroupのIDを指定
    name = owada_name # 登録するPersonの名前を指定
)

kurosaki = face_client.person_group_person.create(
    person_group_id = hanzawa_group.person_group_id, # PersonGroupのIDを指定
    name = kurosaki_name # 登録するPersonの名前を指定
)

print(hanzawa)

{'additional_properties': {}, 'name': None, 'user_data': None, 'person_id': 'c13c5ca2-21ba-4ce1-bb73-7f1ec9460cf2', 'persisted_face_ids': None}


In [522]:
# person idをまとめておく
person_id_dict = {'hanzawa1' : hanzawa, 'hanzawa2' : hanzawa, 'hanzawa3' : hanzawa, 
                  'owada1' : owada, 'owada2' : owada, 'owada3' : owada, 
                  'kurosaki1' : kurosaki, 'kurosaki2' : kurosaki, 'kurosaki3' : kurosaki}
person_id_dict

{'hanzawa1': <azure.cognitiveservices.vision.face.models._models_py3.Person at 0x7ff9145245b0>,
 'hanzawa2': <azure.cognitiveservices.vision.face.models._models_py3.Person at 0x7ff9145245b0>,
 'hanzawa3': <azure.cognitiveservices.vision.face.models._models_py3.Person at 0x7ff9145245b0>,
 'owada1': <azure.cognitiveservices.vision.face.models._models_py3.Person at 0x7ffb9ab21d60>,
 'owada2': <azure.cognitiveservices.vision.face.models._models_py3.Person at 0x7ffb9ab21d60>,
 'owada3': <azure.cognitiveservices.vision.face.models._models_py3.Person at 0x7ffb9ab21d60>,
 'kurosaki1': <azure.cognitiveservices.vision.face.models._models_py3.Person at 0x7ffb9aacd190>,
 'kurosaki2': <azure.cognitiveservices.vision.face.models._models_py3.Person at 0x7ffb9aacd190>,
 'kurosaki3': <azure.cognitiveservices.vision.face.models._models_py3.Person at 0x7ffb9aacd190>}

In [535]:
for key, value in person_id_dict.items():
    print(key, value)

hanzawa1 {'additional_properties': {}, 'name': None, 'user_data': None, 'person_id': 'c13c5ca2-21ba-4ce1-bb73-7f1ec9460cf2', 'persisted_face_ids': None}
hanzawa2 {'additional_properties': {}, 'name': None, 'user_data': None, 'person_id': 'c13c5ca2-21ba-4ce1-bb73-7f1ec9460cf2', 'persisted_face_ids': None}
hanzawa3 {'additional_properties': {}, 'name': None, 'user_data': None, 'person_id': 'c13c5ca2-21ba-4ce1-bb73-7f1ec9460cf2', 'persisted_face_ids': None}
owada1 {'additional_properties': {}, 'name': None, 'user_data': None, 'person_id': 'ce1839b0-a222-4324-bcd0-dd000dd68940', 'persisted_face_ids': None}
owada2 {'additional_properties': {}, 'name': None, 'user_data': None, 'person_id': 'ce1839b0-a222-4324-bcd0-dd000dd68940', 'persisted_face_ids': None}
owada3 {'additional_properties': {}, 'name': None, 'user_data': None, 'person_id': 'ce1839b0-a222-4324-bcd0-dd000dd68940', 'persisted_face_ids': None}
kurosaki1 {'additional_properties': {}, 'name': None, 'user_data': None, 'person_id': '0

#### Faceを登録

- [add_face_from_stream](https://docs.microsoft.com/ja-jp/python/api/azure-cognitiveservices-vision-face/azure.cognitiveservices.vision.face.operations.facelistoperations?view=azure-python#add-face-from-stream-face-list-id--image--user-data-none--target-face-none--detection-model--detection-01---custom-headers-none--raw-false--callback-none----operation-config-)  
    人物グループに人物の顔を追加

In [523]:
n = len(face_path_dict)
keys = []
face_list = []

for i in face_path_dict.keys():
    with open(face_path_dict[i], 'rb') as image:
        keys.append(i)
        face_list.append(face_client.person_group_person.add_face_from_stream(
            person_group_id = hanzawa_group.person_group_id, 
            person_id = person_id_dict[i].person_id, 
            image = image))

In [524]:
# face_listとface_keyを紐付け
face_images_dict = dict(zip(keys, face_list))
face_images_dict

{'hanzawa1': <azure.cognitiveservices.vision.face.models._models_py3.PersistedFace at 0x7ff914144b80>,
 'kurosaki1': <azure.cognitiveservices.vision.face.models._models_py3.PersistedFace at 0x7ff914722f40>,
 'owada1': <azure.cognitiveservices.vision.face.models._models_py3.PersistedFace at 0x7ff914722be0>,
 'hanzawa2': <azure.cognitiveservices.vision.face.models._models_py3.PersistedFace at 0x7ff9140a6d60>,
 'kurosaki2': <azure.cognitiveservices.vision.face.models._models_py3.PersistedFace at 0x7ffb9af7e220>,
 'owada3': <azure.cognitiveservices.vision.face.models._models_py3.PersistedFace at 0x7ffb9c5376d0>,
 'owada2': <azure.cognitiveservices.vision.face.models._models_py3.PersistedFace at 0x7ff9142bdca0>,
 'kurosaki3': <azure.cognitiveservices.vision.face.models._models_py3.PersistedFace at 0x7ff9141b89d0>,
 'hanzawa3': <azure.cognitiveservices.vision.face.models._models_py3.PersistedFace at 0x7ff9142abe50>}

In [525]:
print(face_images_dict['hanzawa1'])
print(face_images_dict['hanzawa2'])
print(face_images_dict['hanzawa3'])

{'additional_properties': {}, 'persisted_face_id': '0e938b2a-b946-4c5f-8181-7804967b6e63', 'user_data': None}
{'additional_properties': {}, 'persisted_face_id': '9125c48e-d04c-49b5-89df-5ba1962d1708', 'user_data': None}
{'additional_properties': {}, 'persisted_face_id': 'd7c6e042-9786-4d0e-b979-67f84dfdb532', 'user_data': None}


#### 最新のperson情報を取得

- [get(person_group_id, person_id, custom_headers=None, raw=False, **operation_config)](https://docs.microsoft.com/ja-jp/python/api/azure-cognitiveservices-vision-face/azure.cognitiveservices.vision.face.operations.persongrouppersonoperations?view=azure-python#get-person-group-id--person-id--custom-headers-none--raw-false----operation-config-)

In [526]:
test = face_client.person_group_person.get(
    person_group_id = hanzawa_group.person_group_id, 
    person_id = person_id_dict['hanzawa1'].person_id)
print(test)

{'additional_properties': {}, 'name': 'hanzawa_naoki', 'user_data': None, 'person_id': 'c13c5ca2-21ba-4ce1-bb73-7f1ec9460cf2', 'persisted_face_ids': ['0e938b2a-b946-4c5f-8181-7804967b6e63', '9125c48e-d04c-49b5-89df-5ba1962d1708', 'd7c6e042-9786-4d0e-b979-67f84dfdb532']}


### 顔写真の判定

In [527]:
detected_faces_dict

{'hanzawa1': [<azure.cognitiveservices.vision.face.models._models_py3.DetectedFace at 0x7ffb21c01520>],
 'kurosaki1': [<azure.cognitiveservices.vision.face.models._models_py3.DetectedFace at 0x7ffb9c743bb0>],
 'owada1': [<azure.cognitiveservices.vision.face.models._models_py3.DetectedFace at 0x7ffb9aacf580>],
 'hanzawa2': [<azure.cognitiveservices.vision.face.models._models_py3.DetectedFace at 0x7ffb9ade19a0>],
 'kurosaki2': [<azure.cognitiveservices.vision.face.models._models_py3.DetectedFace at 0x7ff9141f6a90>],
 'owada3': [<azure.cognitiveservices.vision.face.models._models_py3.DetectedFace at 0x7ffb9ade1760>],
 'owada2': [<azure.cognitiveservices.vision.face.models._models_py3.DetectedFace at 0x7ff9144e2190>],
 'kurosaki3': [<azure.cognitiveservices.vision.face.models._models_py3.DetectedFace at 0x7ff9c3d2da00>],
 'hanzawa3': [<azure.cognitiveservices.vision.face.models._models_py3.DetectedFace at 0x7ff9c3dc8ee0>]}

### 登録済画像と検証用画像の比較

- [verify_face_to_person](https://docs.microsoft.com/ja-jp/python/api/azure-cognitiveservices-vision-face/azure.cognitiveservices.vision.face.operations.faceoperations?view=azure-python#verify-face-to-person-face-id--person-id--person-group-id-none--large-person-group-id-none--custom-headers-none--raw-false----operation-config-)  
    FaceIDとPersonIDを比較する

新しく比較する顔

- [url1：半沢](https://www.crank-in.net/img/db/1406357_650.jpg)
- [url2：大和田](https://pbs.twimg.com/profile_images/378800000485622414/7727f9842f6826183fd0a1e8e58200d1_400x400.jpeg)
- [url3：黒崎](https://s.yimg.jp/images/gyao/trend/syst/2020/082638/01.jpg)

In [528]:
# 検証用の画像の取得
url1 = 'https://www.crank-in.net/img/db/1406357_650.jpg'
url2 = 'https://pbs.twimg.com/profile_images/378800000485622414/7727f9842f6826183fd0a1e8e58200d1_400x400.jpeg'
url3 = 'https://s.yimg.jp/images/gyao/trend/syst/2020/082638/01.jpg'

# 検証用の顔検出
hanzawa_test = face_client.face.detect_with_url(url1)
owada_test = face_client.face.detect_with_url(url2)
kurosaki_test = face_client.face.detect_with_url(url3)

In [537]:
# 検証用画像と半沢のperson idとの比較
valified = face_client.face.verify_face_to_person(
    face_id = hanzawa_test[0].face_id,
    person_group_id = hanzawa_group.person_group_id,
    person_id = person_id_dict['hanzawa1'].person_id
)
print(valified)

{'additional_properties': {}, 'is_identical': True, 'confidence': 0.70687}


In [538]:
# 検証用画像と大和田のperson idとの比較
valified = face_client.face.verify_face_to_person(
    face_id = owada_test[0].face_id,
    person_group_id = hanzawa_group.person_group_id,
    person_id = person_id_dict['owada1'].person_id
)
print(valified)

{'additional_properties': {}, 'is_identical': True, 'confidence': 0.80649}


In [539]:
# 検証用画像と大和田のperson idとの比較
valified = face_client.face.verify_face_to_person(
    face_id = kurosaki_test[0].face_id,
    person_group_id = hanzawa_group.person_group_id,
    person_id = person_id_dict['kurosaki1'].person_id
)
print(valified)

{'additional_properties': {}, 'is_identical': True, 'confidence': 0.75696}


In [542]:
valified.confidence > 0.6

True